In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,007 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,15

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-05-04 13:07:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.11MB/s    in 0.2s    

2023-05-04 13:07:27 (6.11 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://myhope-bucket.s3.amazonaws.com/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [6]:
vine_table= df.select(["star_rating","helpful_votes","total_votes", "vine", "verified_purchase"])
vine_table.show()


+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            3|          4|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|           

In [31]:
df2= df.select(["star_rating","helpful_votes","total_votes", "vine", "verified_purchase"])
df20 = df2.filter(df2["total_votes"]>=20)
df_helpful = df20.filter(df20["helpful_votes"]/ df20["total_votes"]>=0.5)

In [33]:
paid_df = df_helpful.filter(df_helpful["vine"]=="Y")
paid_df.describe().show()
unpaid_df = df_helpful.filter(df_helpful["vine"]=="N")
unpaid_df.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|               463|              463|               463| 463|              463|
|   mean| 4.058315334773218|65.91792656587474|  73.1792656587473|null|             null|
| stddev|1.0562384105884575|94.07596729061886|100.55045599693308|null|             null|
|    min|                 1|               11|                20|   Y|                N|
|    max|                 5|              790|               937|   Y|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+-------

In [34]:
paid_five = paid_df[paid_df["star_rating"]== 5].count()
paid_total = paid_df.count()
paid_percent = paid_five/paid_total
print(paid_five)
print(paid_total)
print(paid_percent)

202
463
0.43628509719222464


In [35]:
unpaid_five = unpaid_df[unpaid_df["star_rating"]== 5].count()
unpaid_total = unpaid_df.count()
unpaid_percent = unpaid_five/unpaid_total
print(unpaid_five)
print(unpaid_total)
print(unpaid_percent)

12033
25094
0.4795170160197657


In [45]:
import pandas as pd 
import numpy as np
import warnings


In [74]:
# Read in the vine_table.csv
vine_table = pd.read_csv("vine_table.csv")
vine_table

,review_id,customer_id,product_id,product_parent,review_date
0,R8EWA1OFT84NX,9970739,B00GSP5D94,329991347,2015-08-31
1,R2JWY4YRQD4FOP,23538442,B00YYDDZGU,646108902,2015-08-31
2,RL5ESX231LZ0B,8273344,B00BUBNZC8,642922361,2015-08-31
3,RRMS9ZWJ2KD08,24557753,B00AWLZFTS,494272733,2015-08-31
4,R14I3ZG5E6S7YM,46263340,B00KM60D3Q,305813185,2015-08-31
...,...,...,...,...,...
1752927,R7W6TPBOZ7WJZ,52873209,B00000J0HR,865087255,1999-07-14
1752928,R1Y4I7KYF7Y5MH,51161415,B00000J0HU,640300757,1999-07-14
1752929,R35M9WTWVWMLX9,52165416,B00000ISE2,455759081,1999-07-13
1752930,R2LUF1DDNNRNAJ,52784607,B00000DMCK,608283258,1999-07-13


In [79]:
vine_df = pd.read_csv("vine_table.csv", index_col=False, header=None)
vine_df.rename({0: "review_id", 1:"star_rating", 2:"helpful_votes", 
                3:"total_votes", 4:"vine", 5:"verified_purchase"}, axis=1, inplace=True)

vine_df.set_index("review_id")

<ipython-input-79-8bd6b91701eb>:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  vine_df = pd.read_csv("vine_table.csv", index_col=False, header=None)


,star_rating,helpful_votes,total_votes,vine
review_id,,,,
review_id,customer_id,product_id,product_parent,review_date
R8EWA1OFT84NX,9970739,B00GSP5D94,329991347,2015-08-31
R2JWY4YRQD4FOP,23538442,B00YYDDZGU,646108902,2015-08-31
RL5ESX231LZ0B,8273344,B00BUBNZC8,642922361,2015-08-31
RRMS9ZWJ2KD08,24557753,B00AWLZFTS,494272733,2015-08-31
...,...,...,...,...
R7W6TPBOZ7WJZ,52873209,B00000J0HR,865087255,1999-07-14
R1Y4I7KYF7Y5MH,51161415,B00000J0HU,640300757,1999-07-14
R35M9WTWVWMLX9,52165416,B00000ISE2,455759081,1999-07-13


In [80]:
vine_votes = (vine_df['vine'].values == 'Y').sum()
non_vine_votes = (vine_df['vine'].values != 'Y').sum()
print(vine_votes, non_vine_votes)

0 1752933


In [96]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where 
# the total_votes count is equal to or greater than 20 to pick reviews that are more 
# likely to be helpful and to avoid having division by zero errors later on.
df20= df.select(["star_rating","helpful_votes","total_votes", "vine", "verified_purchase"])
df20 = df20.filter(df2["total_votes"]>20)
print(df20)

DataFrame[star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [97]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or 
# table that retrieves all the rows where a review was written as part of the 
# Vine program

yes_df = ratio_df[ratio_df['vine'] == 'Y']
print(yes_df)

DataFrame[star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [98]:
# Repeat, but this time retrieve all the rows where the review was not part 
# of the Vine program (unpaid)

no_df = ratio_df[ratio_df['vine'] == 'N']
print(no_df)

DataFrame[star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [101]:
# Determine the total number of reviews, the number of 5-star reviews, and the
# percentage of 5-star reviews for the two types of review (paid vs unpaid).

# Number of reviews and 5* reviews for Y and N, and format output
vine_votes = len(yes_df.index)
yes_votes = ("{:,}").format(vine_votes)
vine_stars = (yes_df['star_rating'].values == 5).sum()
yes_stars = ("{:,}").format(vine_stars)
vine_help = (vine_stars / vine_votes)
yes_perc = "{:.2%}".format(vine_help)
not_vine_votes = len(no_df.index)
not_votes = ("{:,}").format(not_vine_votes)
not_vine_stars = (no_df['star_rating'].values == 5).sum()
not_stars = ("{:,}").format(not_vine_stars)
not_vine_help = (not_vine_stars / not_vine_votes)
not_vine_perc = "{:.2%}".format(not_vine_help)
print(yes_votes, yes_stars, yes_perc, not_votes, not_stars, not_vine_perc)

AttributeError: ignored